In [97]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

In [98]:
train_limpio = pd.read_csv("train_limpio.csv",encoding = "ISO-8859-1")
train_limpio.head(2)

,text,target,@,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad,...,being a,boat capsizes,but if,cant wait,car engulfed,chinaÂÂs stock,city of,collapse at,insultos_por_tweet,expresiones_por_tweet
0,our deeds are the reason of this earthquake ma...,1,0,0,0,1,70,2,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,forest fire near la ronge sask canada,1,0,0,0,1,40,1,0.1,0.4,...,0,0,0,0,0,0,0,0,0,0


In [99]:
test_limpio = pd.read_csv("test_limpio.csv",encoding = "ISO-8859-1")
test_limpio.head(2)

,text,@,cantidad_links,cantidad_signos_pregunta,repeticiones,longitud_tweet,letras_seguidas,sentimiento,objetividad,the,...,being a,boat capsizes,but if,cant wait,car engulfed,chinaÂÂs stock,city of,collapse at,insultos_por_tweet,expresiones_por_tweet
0,just happened a terrible car crash,0,0,0,1,40,2,-1.00,1.00,0,...,0,0,0,0,0,0,0,0,0,0
1,heard about earthquake is different cities sta...,0,0,0,1,70,2,0.25,0.55,0,...,0,0,0,0,0,0,0,0,0,0


In [100]:
target = train_limpio["target"]
target.head(2)

0    1
1    1
Name: target, dtype: int64

In [102]:
df = train_limpio[["text", "target"]]
raw_train_ds, raw_val_ds, raw_test_ds = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

In [103]:
raw_train_ds.head(2)

,text,target
3643,first responders convene for national summit a...,0
5419,southeast dirt riot series crowns champions s...,0


In [104]:
features = raw_train_ds["text"].astype("string")
features.to_numpy(dtype="str")

array(['first responders convene for national summit and awards on gis technology  gisuserpr geotech',
       'southeast dirt riot series crowns champions  southeast dirt riot series crowns championsblackfoot id the so ',
       'devonbreneman hopefully it doesnt electrocute your heated blanket lmao',
       ...,
       'train derailment in patna no news of any casualty so far  the indian express  ',
       'byproduct of metal price meltdown is a higher silver price ',
       'benjidevos thanks thanks '], dtype='<U158')

In [105]:
max_features = 20000
embedding_dim = 128
sequence_length = 280

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

vectorize_layer.adapt(features.to_numpy(dtype="str"))

/home/lucasotero/datos/lib/python3.6/site-packages/tensorflow/python/keras/layers/preprocessing/text_vectorization.py:338: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(preprocessed_data.to_list())


In [107]:
raw_train_ds = tf.data.Dataset.from_tensor_slices((raw_train_ds[["text"]].to_numpy(dtype="str"), raw_train_ds[["target"]].values))
raw_val_ds = tf.data.Dataset.from_tensor_slices((raw_val_ds[["text"]].to_numpy(dtype="str"), raw_val_ds[["target"]].values))
raw_test_ds = tf.data.Dataset.from_tensor_slices((raw_test_ds[["text"]].to_numpy(dtype="str"), raw_test_ds[["target"]].values))

In [108]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

### Esto es el modelo y, aunque parezca lo más dificil, es lo que más se entiende hasta ahora

In [109]:
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [110]:
epochs = 3

# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
4282/4282 [==============================] - 83s 19ms/step - loss: 0.6030 - accuracy: 0.6875 - val_loss: 0.4916 - val_accuracy: 0.7801
Epoch 2/3
4282/4282 [==============================] - 83s 19ms/step - loss: 0.3451 - accuracy: 0.8657 - val_loss: 0.5476 - val_accuracy: 0.7787
Epoch 3/3
4282/4282 [==============================] - 83s 19ms/step - loss: 0.1628 - accuracy: 0.9430 - val_loss: 0.7008 - val_accuracy: 0.7549


In [111]:
model.evaluate(test_ds)

1428/1428 [==============================] - 2s 1ms/step - loss: 0.7827 - accuracy: 0.7199


[0.7826988697052002, 0.7198879718780518]

### Esto ultimo significa que no sabe que predecir :)